In [1]:
import os
import sys
import pandas as pd

In [13]:
VIDEO_DIR = "./../../data/temp/videos/trimmed"
LABEL_FILE = "./../../data/mpgcn_labels.csv"

LABELS = [
    "Transit",
    "Social_People",
    "Play_Object_Normal",
    "Play_Object_Risk",
    "Adult_Assisting",
    "Negative_Contact",
    "No_Activity"
]

In [14]:
videos = pd.read_csv("./../../data/prefiltered_scenes.csv")
videos.head(3)

,video_base,t_start_sec_x,t_end_sec_x,trimmed_name,start_frame12,window_activity,lat,long,timestamp,id,id_person,camera,presence_ratio,mean_edge_touches,mean_motion,duration_sec,keep
0,columpios_cam4-2024-11-21 19:16:11.mp4,6.333333,10.333333,trimmed_columpios_cam4-2024-11-21_19_16_11_00-...,76.0,7.713624,25.653048,-100.285912,2024-11-22 01:16:11.041568+00:00,columpios_cam4-2024-11-21_191611.mp4:addad6d5-...,addad6d5-1fe6-484f-964d-3302e3899325,columpios_cam4,0.516484,0.000000,0.264393,3.797771,True
1,columpioscam2-2024-10-22 22:18:01.mp4,14.500000,18.500000,trimmed_columpioscam2-2024-10-22_22_18_01_00-0...,174.0,7.713624,25.653151,-100.285797,2024-10-23 04:18:01.051273+00:00,columpioscam2-2024-10-22_221801.mp4:a1feb7e2-8...,a1feb7e2-8cb7-4de0-bbc9-117b21756495,columpioscam2,0.604167,0.000000,0.106134,4.005591,True
2,columpioscam1-2024-11-17 17:59:08.mp4,4.333333,8.333333,trimmed_columpioscam1-2024-11-17_17_59_08_00-0...,52.0,7.713624,25.653151,-100.285675,2024-11-17 23:59:08+00:00,columpioscam1-2024-11-17_175908.mp4:58cd563c-e...,58cd563c-eee8-4ee8-835a-250dff0bf206,columpioscam1,1.000000,0.246106,0.712733,4.006154,True


In [15]:
df_label = videos[["trimmed_name", "video_base", "camera", "t_start_sec_x", "t_end_sec_x"]].copy()
df_label["activity_label"] = None
df_label.head()

,trimmed_name,video_base,camera,t_start_sec_x,t_end_sec_x,activity_label
0,trimmed_columpios_cam4-2024-11-21_19_16_11_00-...,columpios_cam4-2024-11-21 19:16:11.mp4,columpios_cam4,6.333333,10.333333,None
1,trimmed_columpioscam2-2024-10-22_22_18_01_00-0...,columpioscam2-2024-10-22 22:18:01.mp4,columpioscam2,14.500000,18.500000,None
2,trimmed_columpioscam1-2024-11-17_17_59_08_00-0...,columpioscam1-2024-11-17 17:59:08.mp4,columpioscam1,4.333333,8.333333,None
3,trimmed_columpioscam1-2024-11-17_17_59_08_00-0...,columpioscam1-2024-11-17 17:59:08.mp4,columpioscam1,4.333333,8.333333,None
4,trimmed_columpioscam3-2024-09-25_12_47_16_00-0...,columpioscam3-2024-09-25 12:47:16.mp4,columpioscam3,11.416667,15.416667,None


### Resume From Previous Progress (if exists)

In [16]:
if os.path.exists(LABEL_FILE):
    existing = pd.read_csv(LABEL_FILE)
    df_label = df_label.merge(existing[["trimmed_name", "activity_label"]],
                              on="trimmed_name", how="left", suffixes=("", "_old"))

    df_label["activity_label"] = df_label["activity_label"].combine_first(df_label["activity_label_old"])
    df_label.drop(columns=["activity_label_old"], inplace=True)
else:
    existing = pd.DataFrame(columns=["trimmed_name", "activity_label"])

### Labeling UI

In [17]:
helpers_path = os.path.abspath(os.path.join(os.getcwd(), "helpers"))
print("Added path:", helpers_path)

if helpers_path not in sys.path:
    sys.path.append(helpers_path)

Added path: /Users/alexeidelgado/Desktop/mpgcn-playground-scenes/script/notebooks/helpers


In [18]:
from script.notebooks.helpers.video_labeling_widget import start_labeling

> ⚠️ Please run this on a web version of the notebook to visualize the video

In [ ]:
start_labeling(df_label, VIDEO_DIR, LABEL_FILE, LABELS)

Output()

> Don't run if videos are not fully labeled yet

In [20]:
class2idx = {label: idx for idx, label in enumerate(LABELS)}
df_label["class_id"] = df_label["activity_label"].map(class2idx)
df_label.to_csv("mpgcn_labels_encoded.csv", index=False)

print(class2idx)

{'Transit': 0, 'Social_People': 1, 'Play_Object_Normal': 2, 'Play_Object_Risk': 3, 'Adult_Assisting': 4, 'Negative_Contact': 5, 'No_Activity': 6}
